In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pulp import *
import pandas as pd
from sklearn.neighbors import DistanceMetric
from math import radians

In [2]:
# Start of with constructing the set of all potential DC sites = I
# Start with only 1 possible DC=DEPOT

#create an index
DC_ID = ["DC_1", "DC_2"] #, 2, 3, 4]

#Gi = fixed cost for establishing depot i
fixed_cost_DC = [5, 5] # 20, 20, 20]

#Maximum Throughput at depot i = Vi
capacity_DC = [20, 20] # 50, 50, 50]

#Position of the DCs
lat_DC = [49.794783, 49.81613668]#, 49.78379436, 49.74101263]
lon_DC = [9.906499, 9.897654496] # 9.896245726, 9.88898842]

dc_tuples = list(zip(DC_ID, fixed_cost_DC, capacity_DC, lat_DC, lon_DC))

set_of_all_DC = pd.DataFrame(dc_tuples, columns = ["DC_ID", "fixed_cost_DC", "capacity_DC", "lat_DC", "lon_DC"])
set_of_all_DC.set_index("DC_ID", inplace = True)
set_of_all_DC

,fixed_cost_DC,capacity_DC,lat_DC,lon_DC
DC_ID,,,,
DC_1,5,20,49.794783,9.906499
DC_2,5,20,49.816137,9.897654


In [3]:
# Define I
I = set_of_all_DC.index.values
I

array(['DC_1', 'DC_2'], dtype=object)

In [4]:
# Construct set of all potential Customers J

# create index
C_ID = ["C1", "C2", "C3", "C4", "C5"]

# Set the demand dj
demand_C = [5, 5, 5, 5, 5]

# Position of customers
lat_C = [49.78981985, 49.79961774, 49.75060158, 49.79996773, 49.76905279]
lon_C = [9.964805881, 9.873297251, 9.919076627, 9.889586523, 9.913514194]

c_tuples = list(zip(C_ID, demand_C, lat_C, lon_C))

set_of_all_customers = pd.DataFrame(c_tuples, columns = ["C_ID", "demand_C", "lat_C", "lon_C"])
set_of_all_customers.set_index("C_ID", inplace = True)
set_of_all_customers

,demand_C,lat_C,lon_C
C_ID,,,
C1,5,49.789820,9.964806
C2,5,49.799618,9.873297
C3,5,49.750602,9.919077
C4,5,49.799968,9.889587
C5,5,49.769053,9.913514


In [5]:
J = set_of_all_customers.index.values
J

array(['C1', 'C2', 'C3', 'C4', 'C5'], dtype=object)

In [6]:
# Construct set of all vehicles K

# create index
V_ID = ["V_1","V_2"]

# Vehicle Capacity Qk
capacity_V = [100,100]

# fixed cost of using Vehicle Fk
fixed_cost_V = [5,5]

v_tuples = list(zip(V_ID, capacity_V, fixed_cost_V))

set_of_all_vehicles = pd.DataFrame(v_tuples, columns = ["V_ID", "capacity_V", "fixed_cost_V"])
set_of_all_vehicles.set_index("V_ID", inplace=True)
set_of_all_vehicles

,capacity_V,fixed_cost_V
V_ID,,
V_1,100,5
V_2,100,5


In [7]:
K = set_of_all_vehicles.index.values
K

array(['V_1', 'V_2'], dtype=object)

In [8]:
#Set up a DF for the distance matrix
data = {"C_ID": ["DC_1","DC_2", # Depot
                "C1","C2","C3","C4","C5"], # Customers
         "lat": [49.794783, 49.81613668, #Depot
                 49.78981985, 49.79961774, 49.75060158, 49.79996773, 49.76905279], #Customers
         "lon": [9.906499, 9.897654496, #Depot
                 9.964805881, 9.873297251, 9.919076627, 9.889586523, 9.913514194]} #Customers
DF = pd.DataFrame(data, columns = ["C_ID", "lat", "lon"])
DF

,C_ID,lat,lon
0,DC_1,49.794783,9.906499
1,DC_2,49.816137,9.897654
2,C1,49.789820,9.964806
3,C2,49.799618,9.873297
4,C3,49.750602,9.919077
5,C4,49.799968,9.889587
6,C5,49.769053,9.913514


In [9]:
DF['lat'] = np.radians(DF['lat'])
DF['lon'] = np.radians(DF['lon'])
DF

,C_ID,lat,lon
0,DC_1,0.869083,0.172901
1,DC_2,0.869456,0.172747
2,C1,0.868996,0.173919
3,C2,0.869167,0.172322
4,C3,0.868312,0.173121
5,C4,0.869173,0.172606
6,C5,0.868634,0.173023


In [10]:
dist = DistanceMetric.get_metric("haversine")

In [11]:
DF[["lat", "lon"]].to_numpy()

array([[0.86908291, 0.17290102],
       [0.86945561, 0.17274666],
       [0.86899629, 0.17391867],
       [0.8691673 , 0.17232155],
       [0.8683118 , 0.17312055],
       [0.8691734 , 0.17260585],
       [0.86863384, 0.17302346]])

In [12]:
#Create the dist matrix in meters
dist_matrix = pd.DataFrame(dist.pairwise(DF[["lat", "lon"]].to_numpy())*6378100, columns=DF.C_ID.unique(), index=DF.C_ID.unique())
dist_matrix.round(1)

,DC_1,DC_2,C1,C2,C3,C4,C5
DC_1,0.0,2460.5,4226.4,2445.7,5000.7,1345.3,2908.3
DC_2,2460.5,0.0,5644.4,2538.4,7456.0,1890.9,5363.8
C1,4226.4,5644.4,0.0,6665.6,5465.2,5522.0,4351.8
C2,2445.7,2538.4,6665.6,0.0,6372.1,1171.1,4464.5
C3,5000.7,7456.0,5465.2,6372.1,0.0,5890.1,2092.6
C4,1345.3,1890.9,5522.0,1171.1,5890.1,0.0,3847.2
C5,2908.3,5363.8,4351.8,4464.5,2092.6,3847.2,0.0


In [13]:
#convert to int
dist_mat1 = dist_matrix.astype(int)
dist_mat1

,DC_1,DC_2,C1,C2,C3,C4,C5
DC_1,0,2460,4226,2445,5000,1345,2908
DC_2,2460,0,5644,2538,7456,1890,5363
C1,4226,5644,0,6665,5465,5521,4351
C2,2445,2538,6665,0,6372,1171,4464
C3,5000,7456,5465,6372,0,5890,2092
C4,1345,1890,5521,1171,5890,0,3847
C5,2908,5363,4351,4464,2092,3847,0


In [14]:
#Test to include from warehouse to warehouse. See if the constraints work then
R123 = [(j,i,k) for j in I for i in I if i!=j for k in K]
R123

[('DC_1', 'DC_2', 'V_1'),
 ('DC_1', 'DC_2', 'V_2'),
 ('DC_2', 'DC_1', 'V_1'),
 ('DC_2', 'DC_1', 'V_2')]

In [15]:
R1 = [(i,j,k) for i in J for j in J if i != j for k in K]
R2 = [(i,j,k) for i in I for j in J if i!=j for k in K]
R3 = [(j,i,k) for j in J for i in I if i!=j for k in K]
R123 = [(j,i,k) for j in I for i in I if i!=j for k in K]
R1 += R2
R1 += R3
R1 += R123
#R1

In [16]:
df3 = pd.DataFrame(R1, columns = ["start", "end","vehicle"])
df3.set_index(["start","end", "vehicle"], inplace=True)
df3.tail(5)

Empty DataFrame
Columns: []
Index: [(C5, DC_2, V_2), (DC_1, DC_2, V_1), (DC_1, DC_2, V_2), (DC_2, DC_1, V_1), (DC_2, DC_1, V_2)]

In [17]:
# Add distances manually, dont know how to do this better
distances_routes2 = [6665.6, 6665.6, 5465.2, 5465.2, 5522.0, 5522.0,4351.8,4351.8,
                    6665.6, 6665.6, 6372.1, 6372.1,	1171.1,1171.1,	4464.5, 4464.5,
                    5465.2, 5465.2,	6372.1, 6372.1, 5890.1, 5890.1,	2092.6, 2092.6,
                    5522.0, 5522.0,	1171.1, 1171.1,	5890.1, 5890.1, 3847.2, 3847.2,
                    4351.8, 4351.8,	4464.5, 4464.5,	2092.6, 2092.6,	3847.2, 3847.2,
                    4226.4, 4226.4,	2445.7, 2445.7,	5000.7, 5000.7,	1345.3, 1345.3,	2908.3, 2908.3,
                    5644.4, 5644.4, 2538.4, 2538.4, 7456.0, 7456.0,	1890.9, 1890.9,	5363.8, 5363.8,
                    4226.4, 4226.4, 5644.4, 5644.4, 2445.7, 2445.7, 2538.4, 2538.4, 5000.7, 5000.7, 7456.0, 7456.0, #C1, C2, C3 - DC_1, DC_2
                    1345.3, 1345.3, 1890.9, 1890.9, 2908.3, 2908.3, 5363.8, 5363.8,
                    2460.5,2460.5, 2460.5,2460.5] #Connection between DCs, might remove later again
                     		
df3["Distance"] = distances_routes2
R_df = df3

In [18]:
R = R_df.index.values
R

array([('C1', 'C2', 'V_1'), ('C1', 'C2', 'V_2'), ('C1', 'C3', 'V_1'),
       ('C1', 'C3', 'V_2'), ('C1', 'C4', 'V_1'), ('C1', 'C4', 'V_2'),
       ('C1', 'C5', 'V_1'), ('C1', 'C5', 'V_2'), ('C2', 'C1', 'V_1'),
       ('C2', 'C1', 'V_2'), ('C2', 'C3', 'V_1'), ('C2', 'C3', 'V_2'),
       ('C2', 'C4', 'V_1'), ('C2', 'C4', 'V_2'), ('C2', 'C5', 'V_1'),
       ('C2', 'C5', 'V_2'), ('C3', 'C1', 'V_1'), ('C3', 'C1', 'V_2'),
       ('C3', 'C2', 'V_1'), ('C3', 'C2', 'V_2'), ('C3', 'C4', 'V_1'),
       ('C3', 'C4', 'V_2'), ('C3', 'C5', 'V_1'), ('C3', 'C5', 'V_2'),
       ('C4', 'C1', 'V_1'), ('C4', 'C1', 'V_2'), ('C4', 'C2', 'V_1'),
       ('C4', 'C2', 'V_2'), ('C4', 'C3', 'V_1'), ('C4', 'C3', 'V_2'),
       ('C4', 'C5', 'V_1'), ('C4', 'C5', 'V_2'), ('C5', 'C1', 'V_1'),
       ('C5', 'C1', 'V_2'), ('C5', 'C2', 'V_1'), ('C5', 'C2', 'V_2'),
       ('C5', 'C3', 'V_1'), ('C5', 'C3', 'V_2'), ('C5', 'C4', 'V_1'),
       ('C5', 'C4', 'V_2'), ('DC_1', 'C1', 'V_1'), ('DC_1', 'C1', 'V_2'),
       ('DC_1', 

In [19]:
model = LpProblem("LRP", LpMinimize)

In [20]:
# new X
x = LpVariable.dicts(name = "x", indexs=([*I,*J],[*I,*J],K), cat = "Binary")
x

{'DC_1': {'DC_1': {'V_1': x_DC_1_DC_1_V_1, 'V_2': x_DC_1_DC_1_V_2},
  'DC_2': {'V_1': x_DC_1_DC_2_V_1, 'V_2': x_DC_1_DC_2_V_2},
  'C1': {'V_1': x_DC_1_C1_V_1, 'V_2': x_DC_1_C1_V_2},
  'C2': {'V_1': x_DC_1_C2_V_1, 'V_2': x_DC_1_C2_V_2},
  'C3': {'V_1': x_DC_1_C3_V_1, 'V_2': x_DC_1_C3_V_2},
  'C4': {'V_1': x_DC_1_C4_V_1, 'V_2': x_DC_1_C4_V_2},
  'C5': {'V_1': x_DC_1_C5_V_1, 'V_2': x_DC_1_C5_V_2}},
 'DC_2': {'DC_1': {'V_1': x_DC_2_DC_1_V_1, 'V_2': x_DC_2_DC_1_V_2},
  'DC_2': {'V_1': x_DC_2_DC_2_V_1, 'V_2': x_DC_2_DC_2_V_2},
  'C1': {'V_1': x_DC_2_C1_V_1, 'V_2': x_DC_2_C1_V_2},
  'C2': {'V_1': x_DC_2_C2_V_1, 'V_2': x_DC_2_C2_V_2},
  'C3': {'V_1': x_DC_2_C3_V_1, 'V_2': x_DC_2_C3_V_2},
  'C4': {'V_1': x_DC_2_C4_V_1, 'V_2': x_DC_2_C4_V_2},
  'C5': {'V_1': x_DC_2_C5_V_1, 'V_2': x_DC_2_C5_V_2}},
 'C1': {'DC_1': {'V_1': x_C1_DC_1_V_1, 'V_2': x_C1_DC_1_V_2},
  'DC_2': {'V_1': x_C1_DC_2_V_1, 'V_2': x_C1_DC_2_V_2},
  'C1': {'V_1': x_C1_C1_V_1, 'V_2': x_C1_C1_V_2},
  'C2': {'V_1': x_C1_C2_V_1, 'V_2'

In [21]:
y = LpVariable.dicts(name = "y", indexs=I, cat = "Binary")
y

{'DC_1': y_DC_1, 'DC_2': y_DC_2}

In [22]:
# new
z = LpVariable.dicts(name = "z", indexs = (I,J), cat = "Binary")
z

{'DC_1': {'C1': z_DC_1_C1,
  'C2': z_DC_1_C2,
  'C3': z_DC_1_C3,
  'C4': z_DC_1_C4,
  'C5': z_DC_1_C5},
 'DC_2': {'C1': z_DC_2_C1,
  'C2': z_DC_2_C2,
  'C3': z_DC_2_C3,
  'C4': z_DC_2_C4,
  'C5': z_DC_2_C5}}

In [23]:
U = [(l,k) for l in J for k in K]
u = LpVariable.dicts(name = "u", indexs = U, lowBound = 0, cat = "Continuous")
u

{('C1', 'V_1'): u_('C1',_'V_1'),
 ('C1', 'V_2'): u_('C1',_'V_2'),
 ('C2', 'V_1'): u_('C2',_'V_1'),
 ('C2', 'V_2'): u_('C2',_'V_2'),
 ('C3', 'V_1'): u_('C3',_'V_1'),
 ('C3', 'V_2'): u_('C3',_'V_2'),
 ('C4', 'V_1'): u_('C4',_'V_1'),
 ('C4', 'V_2'): u_('C4',_'V_2'),
 ('C5', 'V_1'): u_('C5',_'V_1'),
 ('C5', 'V_2'): u_('C5',_'V_2')}

In [24]:
# Objective Function New
# Gi * yi
fixedCost_depot = lpSum([y[i] * set_of_all_DC.loc[i].fixed_cost_DC for i in I])

# Cij * Xijk

variableCosts = lpSum([[x[i][j][k]] * dist_mat1.loc[i,j] for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])

# Fk * Xijk

fixedCost_vehicle = lpSum([[x[i][j][k]] * set_of_all_vehicles.loc[k].fixed_cost_V  for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])

In [25]:
fixedCost_depot

5.0*y_DC_1 + 5.0*y_DC_2 + 0.0

In [26]:
variableCosts

6665*x_C1_C2_V_1 + 6665*x_C1_C2_V_2 + 5465*x_C1_C3_V_1 + 5465*x_C1_C3_V_2 + 5521*x_C1_C4_V_1 + 5521*x_C1_C4_V_2 + 4351*x_C1_C5_V_1 + 4351*x_C1_C5_V_2 + 4226*x_C1_DC_1_V_1 + 4226*x_C1_DC_1_V_2 + 5644*x_C1_DC_2_V_1 + 5644*x_C1_DC_2_V_2 + 6665*x_C2_C1_V_1 + 6665*x_C2_C1_V_2 + 6372*x_C2_C3_V_1 + 6372*x_C2_C3_V_2 + 1171*x_C2_C4_V_1 + 1171*x_C2_C4_V_2 + 4464*x_C2_C5_V_1 + 4464*x_C2_C5_V_2 + 2445*x_C2_DC_1_V_1 + 2445*x_C2_DC_1_V_2 + 2538*x_C2_DC_2_V_1 + 2538*x_C2_DC_2_V_2 + 5465*x_C3_C1_V_1 + 5465*x_C3_C1_V_2 + 6372*x_C3_C2_V_1 + 6372*x_C3_C2_V_2 + 5890*x_C3_C4_V_1 + 5890*x_C3_C4_V_2 + 2092*x_C3_C5_V_1 + 2092*x_C3_C5_V_2 + 5000*x_C3_DC_1_V_1 + 5000*x_C3_DC_1_V_2 + 7456*x_C3_DC_2_V_1 + 7456*x_C3_DC_2_V_2 + 5521*x_C4_C1_V_1 + 5521*x_C4_C1_V_2 + 1171*x_C4_C2_V_1 + 1171*x_C4_C2_V_2 + 5890*x_C4_C3_V_1 + 5890*x_C4_C3_V_2 + 3847*x_C4_C5_V_1 + 3847*x_C4_C5_V_2 + 1345*x_C4_DC_1_V_1 + 1345*x_C4_DC_1_V_2 + 1890*x_C4_DC_2_V_1 + 1890*x_C4_DC_2_V_2 + 4351*x_C5_C1_V_1 + 4351*x_C5_C1_V_2 + 4464*x_C5_C2_V_1 +

In [27]:
model += fixedCost_depot + variableCosts + fixedCost_vehicle

In [28]:
# new (2)
for j in J:
    model += lpSum([[x[i][j][k]] for i in [*I,*J] for k in K if i!=j]) == 1

In [29]:
# (3) new
for k in K:
    model += lpSum(set_of_all_customers.loc[j].demand_C for j in J) * lpSum([[x[i][j][k] for i in [*I,*J] for j in J if i!=j]]) <= set_of_all_vehicles.loc[k].capacity_V

In [30]:
# (4) new
for l in J:
    for j in J:
        if l!=j:
            for k in K:
                model += (u[l,k] - u[j,k] + (len(set_of_all_customers) * [x[l][j][k]])) <= len(set_of_all_customers) - 1

In [31]:
# (5) new - works since we defined DC to DC connections in Xijk
for i in [*I,*J]:
    for k in K:
        model+= lpSum([[x[i][j][k]] for j in [*I,*J] if i!=j]) - lpSum([[x[j][i][k]] for j in [*I,*J] if i!=j]) == 0

In [32]:
# (6) new
for k in K:
    model += lpSum([[x[i][j][k]] for i in I for j in J]) <= 1

In [33]:
# (7) new
for i in I:
    model += lpSum(z[i][j] * set_of_all_customers.loc[j].demand_C for j in J) - (set_of_all_DC.loc[i].capacity_DC * y[i]) <= 0

In [34]:
# (8) new:
#for i in I:
#    for j in J:
#        for k in K:
#            model += lpSum([x[i][u][k] + [x[u][j][k]] for u in [*I,*J] if u!=i if u!=j]) - z[i][j] <= 1

In [35]:
# (8) new: Removed if's
for i in I:
    for j in J:
        for k in K:
            model += lpSum([x[i][u][k] + [x[u][j][k]] for u in [*I,*J]]) - z[i][j] <= 1

In [36]:
#model

In [37]:
model.solve()
LpStatus[model.status]

'Optimal'

In [39]:
for variable in model.variables():
    print("{} = {}".format(variable.name, variable.varValue))

u_('C1',_'V_1') = 0.0
u_('C1',_'V_2') = 0.0
u_('C2',_'V_1') = 0.0
u_('C2',_'V_2') = 1.0
u_('C3',_'V_1') = 1.0
u_('C3',_'V_2') = 0.0
u_('C4',_'V_1') = 0.0
u_('C4',_'V_2') = 0.0
u_('C5',_'V_1') = 4.0
u_('C5',_'V_2') = 0.0
x_C1_C1_V_1 = 0
x_C1_C1_V_2 = 0
x_C1_C2_V_1 = 0
x_C1_C2_V_2 = 0
x_C1_C3_V_1 = 1
x_C1_C3_V_2 = 0
x_C1_C4_V_1 = 0
x_C1_C4_V_2 = 0
x_C1_C5_V_1 = 0
x_C1_C5_V_2 = 0
x_C1_DC_1_V_1 = 0
x_C1_DC_1_V_2 = 0
x_C1_DC_2_V_1 = 0
x_C1_DC_2_V_2 = 0
x_C2_C1_V_1 = 0
x_C2_C1_V_2 = 0
x_C2_C2_V_1 = 0
x_C2_C2_V_2 = 0
x_C2_C3_V_1 = 0
x_C2_C3_V_2 = 0
x_C2_C4_V_1 = 0
x_C2_C4_V_2 = 0
x_C2_C5_V_1 = 0
x_C2_C5_V_2 = 0
x_C2_DC_1_V_1 = 0
x_C2_DC_1_V_2 = 0
x_C2_DC_2_V_1 = 0
x_C2_DC_2_V_2 = 1
x_C3_C1_V_1 = 0
x_C3_C1_V_2 = 0
x_C3_C2_V_1 = 0
x_C3_C2_V_2 = 0
x_C3_C3_V_1 = 0
x_C3_C3_V_2 = 0
x_C3_C4_V_1 = 0
x_C3_C4_V_2 = 0
x_C3_C5_V_1 = 1
x_C3_C5_V_2 = 0
x_C3_DC_1_V_1 = 0
x_C3_DC_1_V_2 = 0
x_C3_DC_2_V_1 = 0
x_C3_DC_2_V_2 = 0
x_C4_C1_V_1 = 0
x_C4_C1_V_2 = 0
x_C4_C2_V_1 = 0
x_C4_C2_V_2 = 1
x_C4_C3_V_1 = 0
x_C4

In [40]:
# new cost printer, without using the R_df for distances
fixedC = sum([y[i].varValue * set_of_all_DC.loc[i].fixed_cost_DC for i in I])
#varC = sum([x[i][j][k].varValue * dist_mat1.loc[i,j] for i,j,k in R])
varC = sum([x[i][j][k].varValue *  dist_mat1.loc[i,j] for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])
fixedCV = sum([x[i][j][k].varValue * set_of_all_vehicles.loc[k].fixed_cost_V  for i,j,k in R])
print("Distance traveled: {}m \nFixed Costs for Depots: {}€ \nFixed Cost for Vehicles: {}€".format(varC, fixedC, fixedCV))

Distance traveled: 20290m 
Fixed Costs for Depots: 10.0€ 
Fixed Cost for Vehicles: 35€


In [41]:
# Add some solution printer for which DCs are opened
print("The following DCs are established:")
for i in I:
    if y[i].varValue >= 0.1:
        print("-{}".format(set_of_all_DC.loc[i].name)) 

The following DCs are established:
-DC_1
-DC_2


In [42]:
# Decision Varibale Overview
for variable in model.variables():
    if variable.varValue >= 0.1:
        print("{} = {}".format(variable.name, variable.varValue))

u_('C2',_'V_2') = 1.0
u_('C3',_'V_1') = 1.0
u_('C5',_'V_1') = 4.0
x_C1_C3_V_1 = 1
x_C2_DC_2_V_2 = 1
x_C3_C5_V_1 = 1
x_C4_C2_V_2 = 1
x_C5_DC_1_V_1 = 1
x_DC_1_C1_V_1 = 1
x_DC_2_C4_V_2 = 1
y_DC_1 = 1
y_DC_2 = 1
z_DC_1_C1 = 1
z_DC_1_C3 = 1
z_DC_1_C5 = 1
z_DC_2_C2 = 1
z_DC_2_C4 = 1


In [43]:
# new route count printer
Route_v1 = []
Route_v2 = []
v1 = sum([x[i][j][k].varValue for i,j,k in R1 if k == ("V_1")])
v2 = sum([x[i][j][k].varValue for i,j,k in R1 if k == ("V_2")])
Route_v1.append(v1)
Route_v2.append(v2)
print("Vehicle 1 has to drive {} routes and Vehicle 2 has to drive {} routes to satisfy all demand".format(Route_v1, Route_v2))

Vehicle 1 has to drive [4] routes and Vehicle 2 has to drive [3] routes to satisfy all demand


In [44]:
#This can print the total routes needed but not how many each vehicle drove

Route_Counter = sum([x[i][j][k].varValue for i,j,k in R1])
print("The Vehicles have to drive {} routes to satisfy all demand".format(Route_Counter))

The Vehicles have to drive 7 routes to satisfy all demand


In [45]:
# new route printer (not using the R_df for distances anymore)
print("The following Routes have to be driven to satisfy all demand:")
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1:
        print("From {} to {} in Vehicle {} for length {}m".format(i,j,k, dist_mat1.loc[i,j]))

The following Routes have to be driven to satisfy all demand:
From C1 to C3 in Vehicle V_1 for length 5465m
From C3 to C5 in Vehicle V_1 for length 2092m
From C4 to C2 in Vehicle V_2 for length 1171m
From DC_1 to C1 in Vehicle V_1 for length 4226m
From DC_2 to C4 in Vehicle V_2 for length 1890m
From C2 to DC_2 in Vehicle V_2 for length 2538m
From C5 to DC_1 in Vehicle V_1 for length 2908m


In [46]:
# Route Order Printer - still WIP since i and k are hardcoded here. Would be nice to do this without having to call specific Warehouses/Vehicles since the Model will grow in size substantially
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and i == ("DC_1") and k == ("V_1"):
        print("Vehicle {} starts from {} and drives to {} with route length {}m".format(k,i,j, dist_mat1.loc[i,j]))
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and k == ("V_1") and i != ("DC_1"):
        print("Then drives from {} to {} with route length {}m".format(i,j, dist_mat1.loc[i,j]))

Vehicle V_1 starts from DC_1 and drives to C1 with route length 4226m
Then drives from C1 to C3 with route length 5465m
Then drives from C3 to C5 with route length 2092m
Then drives from C5 to DC_1 with route length 2908m


In [50]:
# Route Order Printer minimalistic Vehicle 1
print("Vehicle 1 drives the following route to satisfy all demand:")
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and i == ("DC_1") and k == ("V_1"):
        print("{} -> {}: route length {}m".format(i,j, dist_mat1.loc[i,j]))
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and k == ("V_1") and i != ("DC_1"):
        print(" {} -> {}: route length {}m".format(i,j, dist_mat1.loc[i,j]))
        #print("For a total vehicle route length {}m".format(sum(dist_mat1.loc[i,j])))

Vehicle 1 drives the following route to satisfy all demand:
DC_1 -> C1: route length 4226m
 C1 -> C3: route length 5465m
 C3 -> C5: route length 2092m
 C5 -> DC_1: route length 2908m


In [49]:
# Route Order Printer minimalistic Vehicle 2
print("Vehicle 2 drives the following route to satisfy all demand:")
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and i == ("DC_2") and k == ("V_2"):
        print("{} -> {}: route length {}m".format(i,j, dist_mat1.loc[i,j]))
for i,j,k in R:
    if x[i][j][k].varValue >= 0.1 and k == ("V_2") and i != ("DC_2"):
        print(" {} -> {}: route length {}m".format(i,j, dist_mat1.loc[i,j]))
        #print("For a total vehicle route length {}m".format(sum(dist_mat1.loc[i,j])))

Vehicle 2 drives the following route to satisfy all demand:
DC_2 -> C4: route length 1890m
 C4 -> C2: route length 1171m
 C2 -> DC_2: route length 2538m


<font size=6>**Idea to add Variable Warehousing Cost into the Objective Function**</font>

![Wu_New](https://i.imgur.com/DeVQzoi.png)

VCi being the respective Variable Warehousing Cost (Picking) in each Warehouse<br>
For code implementation just add a cost factor to the DC Dataframe <br>
VCi = set_of_all_DC.loc[i].varCost (example)


In [54]:
# Start of with constructing the set of all potential DC sites = I
# Start with only 1 possible DC=DEPOT

#create an index
DC_ID = ["DC_1", "DC_2"] #, 2, 3, 4]

#Gi = fixed cost for establishing depot i
fixed_cost_DC = [20, 40] # 20, 20, 20]

#Maximum Throughput at depot i = Vi
capacity_DC = [50, 200] # 50, 50, 50]

#Variable Warehousing Cost (Picking)
varCost_DC = [12,15]

#Position of the DCs
lat_DC = [49.794783, 49.81613668]#, 49.78379436, 49.74101263]
lon_DC = [9.906499, 9.897654496] # 9.896245726, 9.88898842]

dc_tuples = list(zip(DC_ID, fixed_cost_DC, capacity_DC, varCost_DC, lat_DC, lon_DC))

set_of_all_DC = pd.DataFrame(dc_tuples, columns = ["DC_ID", "fixed_cost_DC", "capacity_DC", "varCost_DC", "lat_DC", "lon_DC"])
set_of_all_DC.set_index("DC_ID", inplace = True)
set_of_all_DC

,fixed_cost_DC,capacity_DC,varCost_DC,lat_DC,lon_DC
DC_ID,,,,,
DC_1,20,50,12,49.794783,9.906499
DC_2,40,200,15,49.816137,9.897654


In [76]:
# Objective Function New
# Gi * yi
fixedCost_depot = lpSum([y[i] * set_of_all_DC.loc[i].fixed_cost_DC for i in I])

# Cij * Xijk

variableCosts = lpSum([[x[i][j][k]] * dist_mat1.loc[i,j] for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])

#VCi * dj*zj ; not sure if this is correct, have to play around 
#variableCosts_DC = lpSum(set_of_all_DC.loc[i].varCost_DC for i in I) * lpSum([z[i][j] * set_of_all_customers.loc[j].demand_C for j in J for i in I])
variableCosts_DC = lpSum(set_of_all_DC.loc[i].varCost_DC * z[i][j] * set_of_all_customers.loc[j].demand_C for j in J for i in I)
# Fk * Xijk
fixedCost_vehicle = lpSum([[x[i][j][k]] * set_of_all_vehicles.loc[k].fixed_cost_V  for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])

In [77]:
# This cant be right, Sum Parameter does something funky here
variableCosts_DC

60.0*z_DC_1_C1 + 60.0*z_DC_1_C2 + 60.0*z_DC_1_C3 + 60.0*z_DC_1_C4 + 60.0*z_DC_1_C5 + 75.0*z_DC_2_C1 + 75.0*z_DC_2_C2 + 75.0*z_DC_2_C3 + 75.0*z_DC_2_C4 + 75.0*z_DC_2_C5 + 0.0

In [58]:
fixedCost_depot

20.0*y_DC_1 + 40.0*y_DC_2 + 0.0

In [59]:
fixedCost_vehicle

5*x_C1_C2_V_1 + 5*x_C1_C2_V_2 + 5*x_C1_C3_V_1 + 5*x_C1_C3_V_2 + 5*x_C1_C4_V_1 + 5*x_C1_C4_V_2 + 5*x_C1_C5_V_1 + 5*x_C1_C5_V_2 + 5*x_C1_DC_1_V_1 + 5*x_C1_DC_1_V_2 + 5*x_C1_DC_2_V_1 + 5*x_C1_DC_2_V_2 + 5*x_C2_C1_V_1 + 5*x_C2_C1_V_2 + 5*x_C2_C3_V_1 + 5*x_C2_C3_V_2 + 5*x_C2_C4_V_1 + 5*x_C2_C4_V_2 + 5*x_C2_C5_V_1 + 5*x_C2_C5_V_2 + 5*x_C2_DC_1_V_1 + 5*x_C2_DC_1_V_2 + 5*x_C2_DC_2_V_1 + 5*x_C2_DC_2_V_2 + 5*x_C3_C1_V_1 + 5*x_C3_C1_V_2 + 5*x_C3_C2_V_1 + 5*x_C3_C2_V_2 + 5*x_C3_C4_V_1 + 5*x_C3_C4_V_2 + 5*x_C3_C5_V_1 + 5*x_C3_C5_V_2 + 5*x_C3_DC_1_V_1 + 5*x_C3_DC_1_V_2 + 5*x_C3_DC_2_V_1 + 5*x_C3_DC_2_V_2 + 5*x_C4_C1_V_1 + 5*x_C4_C1_V_2 + 5*x_C4_C2_V_1 + 5*x_C4_C2_V_2 + 5*x_C4_C3_V_1 + 5*x_C4_C3_V_2 + 5*x_C4_C5_V_1 + 5*x_C4_C5_V_2 + 5*x_C4_DC_1_V_1 + 5*x_C4_DC_1_V_2 + 5*x_C4_DC_2_V_1 + 5*x_C4_DC_2_V_2 + 5*x_C5_C1_V_1 + 5*x_C5_C1_V_2 + 5*x_C5_C2_V_1 + 5*x_C5_C2_V_2 + 5*x_C5_C3_V_1 + 5*x_C5_C3_V_2 + 5*x_C5_C4_V_1 + 5*x_C5_C4_V_2 + 5*x_C5_DC_1_V_1 + 5*x_C5_DC_1_V_2 + 5*x_C5_DC_2_V_1 + 5*x_C5_DC_2_V_2 + 

In [78]:
model += fixedCost_depot + variableCosts + variableCosts_DC + fixedCost_vehicle

In [ ]:
model

In [79]:
model.solve()
LpStatus[model.status]

'Optimal'

In [80]:
# Add some solution printer for which DCs are opened
print("The following DCs are established:")
for i in I:
    if y[i].varValue >= 0.1:
        print("-{}".format(set_of_all_DC.loc[i].name)) 

The following DCs are established:
-DC_1
-DC_2


In [81]:
# Decision Varibale Overview
for variable in model.variables():
    if variable.varValue >= 0.1:
        print("{} = {}".format(variable.name, variable.varValue))

u_('C3',_'V_2') = 1.0
u_('C4',_'V_1') = 1.0
u_('C5',_'V_2') = 4.0
x_C1_C3_V_2 = 1
x_C2_C4_V_1 = 1
x_C3_C5_V_2 = 1
x_C4_DC_2_V_1 = 1
x_C5_DC_1_V_2 = 1
x_DC_1_C1_V_2 = 1
x_DC_2_C2_V_1 = 1
y_DC_1 = 1
y_DC_2 = 1
z_DC_1_C1 = 1
z_DC_1_C3 = 1
z_DC_1_C5 = 1
z_DC_2_C2 = 1
z_DC_2_C4 = 1


In [83]:
# new cost printer, without using the R_df for distances
fixedC = sum([y[i].varValue * set_of_all_DC.loc[i].fixed_cost_DC for i in I])
#varC = sum([x[i][j][k].varValue * dist_mat1.loc[i,j] for i,j,k in R])
varC = sum([x[i][j][k].varValue *  dist_mat1.loc[i,j] for i in [*I,*J] for j in [*I,*J] for k in K if i!=j])
fixedCV = sum([x[i][j][k].varValue * set_of_all_vehicles.loc[k].fixed_cost_V  for i,j,k in R])
varDC = sum(set_of_all_DC.loc[i].varCost_DC * z[i][j].varValue * set_of_all_customers.loc[j].demand_C for j in J for i in I)
print("Distance traveled: {}m \nFixed Costs for Depots: {}€ \nFixed Cost for Vehicles: {}€ \n Variable Warehousing Cost {}€".format(varC, fixedC, fixedCV, varDC))

Distance traveled: 20290m 
Fixed Costs for Depots: 60.0€ 
Fixed Cost for Vehicles: 35€ 
 Variable Warehousing Cost 330.0€


Var warehousing Cost should be 330€

3 Customers allocated to DC1 = 3 x 5 x 12 = 180<br>
2 Customers allocated to DC2 = 2 x 5 x 15 = 150<br>
                                150+180 = 330<br>
                                675 is just 5x 135 from the formula since we have 5 customers allocated to DCs